In [40]:
import csv
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import random

import numpy as np
from datasets import load_metric

In [2]:
# model loading
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [29]:
# data loading, might take a bit
data = csv.reader(open("./clues_multitags.csv", 'r'))
cluesTags = []

for l in data:
    cluesTags.append((l[2], l[4]))

random.shuffle(cluesTags)
clues, tags = zip(*cluesTags)
clues = list(clues[:10000])
tags = list(tags[:10000])

In [34]:
clues = list(clues[:10000])
tags = list(tags[:10000])

In [35]:
# prepare 10000 token vectors
encoded_clues = tokenizer(clues[:10000], padding=True, truncation=True, return_tensors='pt')

In [42]:
# train model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=1)

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")
metric = load_metric("f1")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_clues[:90000],
    eval_dataset=encoded_clues[:90000],
    compute_metrics=metric)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [43]:
trainer.train()

/home/henry/.local/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 10000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3750


TypeError: can only join an iterable

In [ ]:
frimo datasets import load_dataset